In [2]:
import urllib.request as rq
import urllib.parse as parse
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup

In [44]:
## 연습!
# https://finance.naver.com/ 에 접속하여 달러 환율정보 확인

url = 'https://finance.naver.com/'

res = rq.urlopen(url).read()
data = res.decode("euc-kr")

soup = BeautifulSoup(data, "html.parser")

price = soup.select_one("div.group1 > table.tbl_home > tbody > tr.up.bold > td")
print(f"미국 환율 : {price.string} 원")

미국 환율 : 1,273.20 원


In [143]:
## 거래 상위 10개를 추출하여 출력하세요
## 종목과 가격 => 대한전기 2140와 같은 식으로 출력

top_deal_names = soup.select("#_topItems1 > tr > th")
name_lst = []
for i in range(len(top_deal_names)):
    name_lst.append(top_deal_names[i].text)
top_deal_prices = soup.select("#_topItems1 > tr > td")
price_lst = []
top_deal_prices
for i in range(0,len(top_deal_prices),3):
    price_lst.append(top_deal_prices[i].text)
    
    
print("\t\t종목\t\t\t  가격")
for j in range(10) :
    if len(name_lst[j]) > 13 and len(name_lst[j]) <= 16:
        print(f"{j+1}위 {name_lst[j]} \t\t {price_lst[j]} 원")
    elif len(name_lst[j]) >= 15:
        print(f"{j+1}위 {name_lst[j]} \t {price_lst[j]} 원")
    else:
        print(f"{j+1}위 {name_lst[j]} \t\t\t\t {price_lst[j]} 원")

# top_list = soup.select("#_topItems1 > tr > th > a")
# top_list_name = [top_list[i].string for i in range(10)]
# top_list_pr = soup.select("#_topItems1 > tr > td")
# top_list_pr1 = [top_list_pr[i].string for i in range(0,30,3)] # 가격
# top_list_pr2 = [top_list_pr[i].string for i in range(1,30,3)] # 상승 or 하락
# top_list_pr3 = [top_list_pr[i].string for i in range(2,30,3)] # 등락률

		종목			  가격
1위 대한전선 				 2,095 원
2위 KODEX 200선물인버스2X 		 2,665 원
3위 부산주공 				 909 원
4위 GS글로벌 				 4,670 원
5위 삼성 인버스 2X WTI원유 선물 ETN 	 125 원
6위 소니드 				 1,360 원
7위 이루온 				 2,900 원
8위 신한 인버스 2X WTI원유 선물 ETN(H) 	 120 원
9위 TYM 				 3,520 원
10위 휴림로봇 				 3,980 원


In [152]:
## 링크에 있는 내용을 한꺼번에 처리하기

# 재귀적으로 탐색하기 위해서 사용하는 함수 - urljoin(base, "상대경로") 함수
from urllib.parse import urljoin

# 상대경로
base = "http://example.com/html/a.html"
print(urljoin(base, "b.html"))              # http://example.com/html/b.html
print(urljoin(base, "sub/c.html"))          # http://example.com/html/sub/c.html
print(urljoin(base, "../index.html"))       # http://example.com/index.html
print(urljoin(base, "../img/home.png"))     # http://example.com/img/home.png
print(urljoin(base, "../css/home.css"))     # http://example.com/css/home.css

# 주의사항(절대경로)
print(urljoin(base, "/index.html"))         # http://example.com/index.html
print(urljoin(base, "http://otherExample.com/wiki"))    # http://otherExample.com/wiki
print(urljoin(base, "//otherExample.com/wiki"))         # http://otherExample.com/wiki

# 참고
url = "http://example.com/html/a.html"
t = urlparse(url)
t.netloc                # 'example.com'
t.path                  # '/html/a.html'


http://example.com/html/b.html
http://example.com/html/sub/c.html
http://example.com/index.html
http://example.com/img/home.png
http://example.com/css/home.css
http://example.com/index.html
http://otherExample.com/wiki
http://otherExample.com/wiki


In [ ]:
## 처리 순서
# 1. HTML분석
# 2. 링크 추출
# 3. 각 링크 대상에 대해 다음 동작 구동
#   3-1. 다운로드 파일인 경우 다운로드(img, css..등등)
#   3-2. 다운로드 파일이 아닌 경우(파일 HTML이라면 1번으로 돌아가 다시 실행)

In [ ]:
## 모든 페이지를 한꺼번에 다운받는 프로그램
# (https://docs.python.org/3.10/library/index.html)
# 
# 모듈
from bs4 import BeautifulSoup
from urllib.request import *
from urllib.parse import *
from os import makedirs
import os.path, time, re

## 처리한 내용이 파일인지 여부 확인을 위한 변수
proc_files = {}

# HTML 내부에 있는 링크를 추출하는 함수
def enum_links(html, base):
    soup = BeautifulSoup(html, "html.parser")
    links = soup.select("link[rel='stylesheet']")   # CSS
    links += soup.select("a[href]")                 # 링크
    
    result = []
    # href 속성을 추출하고, 링크를 절대경로로 변환하여 저장
    for a in links:
        href = a.attrs['href']
        url = urljoin(base, href)
        result.append(url)
    return result

# 파일을 다운로드하고 저장하는 함수
def download_file(url):
    o = urlparse(url)
    savepath = "./" + o.netloc + o.path
    if re.search(r"/$", savepath):          # savepath의 마지막이 "/"라면..
        savepath += "index.html"            # 기본문서 처리
    
    # 디렉토리를 만드는 작업
    savedir = os.path.dirname(savepath)     # savepath의 경로를 추출
    
    # 모두 다운됐는지 여부 확인
    if os.path.exists(savepath) : return savepath
    
    # 다운받을 디렉토리를 생성
    if not os.path.exists(savedir):
        print("mkdir = " , savedir)
        makedirs(savedir)
    
    # 파일 다운받기
    try:
        print("download = " , url)
        urlretrieve(url, savepath)
        time.sleep(1)       # 1초 쉬었다가 동작
        return savepath
    except:
        print("다운로드 실패 : ", url)
        return None

# HTML을 분석하고 다운받는 함수
def analyze_html(url, root_url):
    savepath = download_file(url)
    if savepath is None : return
    if savepath in proc_files: return     # 이미 처리되었다면 실행하지 않음
    proc_files[savepath] = True
    print("analyze_html = ", url)
    
    # 링크추출
    html = open(savepath, "r", encoding="utf-8").read()
    links = enum_links(html, url)
    
    for link_url in links:
        # 링크가 루트 이외의 경로를 나타낸다면 무시
        if link_url.find(root_url) != 0:
            if not re.search(r".css$", link_url) : continue
            
        # HTML 이라면
        if re.search(r".(html|htm)$", link_url) :
            # 재귀적으로 html을 분석
            analyze_html(link_url, root_url)
            continue
        
        # 기타 파일
        download_file(link_url)

# 일반 파이썬 파일인 경우 사용
# if __name__ == "__main__":
#     # url에 있는 모든 것 다운받기
#     url = "https://docs.python.org/3.10/library/"
#     analyze_html(url, url)

# 실행
url = "https://docs.python.org/3.10/library/"
analyze_html(url, url)
    

In [ ]:
### requests 패키지를 이용한 접근
### 파이썬으로 로그인하기
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# 아이디와 비밀번호 지정
USER = "ojy033"
PASS = "dhwndud951753!"

# 세션 시작
session = requests.session()
# 로그인하기
login_info = {
    "m_id" : USER,
    "m_passwd" : PASS
}

url_login = "https://www.hanbit.co.kr/member/login.html"
res = session.post(url_login, data=login_info)
res.raise_for_status() # 오류가 발생하면 예외를 발생

# 마이페이지에 접근
url_mypage = "https://www.hanbit.co.kr/myhanbit/myhanbit.html"
res = session.get(url_mypage)
print(res.text)

In [ ]:
## requests 모듈의 메서드

# get요청
r = requests.get("http://google.com")

# post요청
formdata = {
    "key1" : "value1",
    "key2" : "value2"
}

r = requests.post("http://example.com", data = formdata)
r.content

In [ ]:
# 업로드
r = requests.put("http://httpbin.org/put")
# 삭제
r = requests.delete("http://httpbin.org/delete")
# header정보만 받아오기
r = requests.head("http://httpbin.org/get")

In [ ]:
r = requests.put("http://localhost:8100/put.txt", data = "test")

In [ ]:
from http.server import HTTPServer, SimpleHTTPRequestHandler

server = HTTPServer(('0.0.0.0', 8100), SimpleHTTPRequestHandler)
server.serve_forever()

In [174]:
from selenium import webdriver
USER = "ojy033"
PASS = "dhwndud951753!"
path = "c:\\webdriver\\chromedriver.exe"
driver = webdriver.Chrome(path)

C:\Users\user\AppData\Local\Temp\ipykernel_3536\144402326.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [175]:
# 한빛로그인 페이지
url_login = "https://www.hanbit.co.kr/member/login.html"
driver.get(url_login)

In [176]:
# 로그인 창에 ID와 비밀번호 입력
# USER = "wntpdls"
# PASS = "wntpdls95"
USER = "ojy033"
PASS = "dhwndud951753!"
m_id = driver.find_element_by_id('m_id')
m_id.clear()
m_id.send_keys(USER)
m_passwd = driver.find_element_by_id('m_passwd')
m_passwd.clear()
m_passwd.send_keys(PASS)


C:\Users\user\AppData\Local\Temp\ipykernel_3536\3070985115.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  m_id = driver.find_element_by_id('m_id')
C:\Users\user\AppData\Local\Temp\ipykernel_3536\3070985115.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  m_passwd = driver.find_element_by_id('m_passwd')


In [177]:
# 로그인 버튼 클릭
driver.find_element_by_class_name('btn_login').click()

C:\Users\user\AppData\Local\Temp\ipykernel_3536\695084030.py:2: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name('btn_login').click()


In [178]:
driver.get("https://www.hanbit.co.kr/myhanbit/myhanbit.html")

In [179]:
mileage = driver.find_element_by_css_selector(".mileage_section1 span")
ecoin = driver.find_element_by_css_selector(".mileage_section2 span")
print(f"한빛 마일리지 : {mileage.text} 점")
print(f"한빛 이코인 : {ecoin.text} 점")

한빛 마일리지 : 2,000 점
한빛 이코인 : 0 점


C:\Users\user\AppData\Local\Temp\ipykernel_3536\3513169643.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  mileage = driver.find_element_by_css_selector(".mileage_section1 span")
C:\Users\user\AppData\Local\Temp\ipykernel_3536\3513169643.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  ecoin = driver.find_element_by_css_selector(".mileage_section2 span")


In [183]:
naver = "https://www.naver.com/"
driver.get(naver)

In [184]:
# 로그인 버튼 클릭
driver.find_element_by_class_name('link_login').click()

C:\Users\user\AppData\Local\Temp\ipykernel_3536\3314414918.py:2: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name('link_login').click()


In [189]:
USER = "ojy033"
PASS = "wndud951"
id = driver.find_element_by_id('id')
id.clear()
id.send_keys(USER)
pw = driver.find_element_by_id('pw')
pw.clear()
pw.send_keys(PASS)

C:\Users\user\AppData\Local\Temp\ipykernel_3536\2112023443.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  id = driver.find_element_by_id('id')
C:\Users\user\AppData\Local\Temp\ipykernel_3536\2112023443.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pw = driver.find_element_by_id('pw')


In [190]:
driver.find_element_by_class_name('btn_login').click()

C:\Users\user\AppData\Local\Temp\ipykernel_3536\3056327959.py:1: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name('btn_login').click()


In [191]:
pw = driver.find_element_by_id('pw')
pw.clear()
pw.send_keys(PASS)

C:\Users\user\AppData\Local\Temp\ipykernel_3536\663219741.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pw = driver.find_element_by_id('pw')


In [192]:
captcha = driver.find_element_by_id('captcha')
captcha.clear()
captcha.send_keys('36000')

C:\Users\user\AppData\Local\Temp\ipykernel_3536\2175033482.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  captcha = driver.find_element_by_id('captcha')


In [193]:
driver.find_element_by_class_name('btn_login').click()

C:\Users\user\AppData\Local\Temp\ipykernel_3536\3056327959.py:1: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name('btn_login').click()


In [195]:
url_pay = "https://order.pay.naver.com/home"
driver.get(url_pay)

In [204]:
npoint = driver.find_element_by_css_selector(".my_npoint dd")
print(f"네이버포인트 : {npoint.text}")

네이버포인트 : 556원


C:\Users\user\AppData\Local\Temp\ipykernel_3536\3248183618.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  npoint = driver.find_element_by_css_selector(".my_npoint dd")


In [4]:
import pyperclip
from selenium.webdriver.common.keys import Keys
import time
path = "c:\\webdriver\\chromedriver.exe"
driver = webdriver.Chrome(path)
naver = "https://www.naver.com/"
driver.get(naver)
driver.find_element_by_class_name('link_login').click()
USER = "ojy033"
PASS = "wndud951"
id = driver.find_element_by_id('id')
id.click()
pyperclip.copy(USER)
id.send_keys(Keys.CONTROL, 'v')
time.sleep(2)
pw = driver.find_element_by_id('pw')
pw.click()
pyperclip.copy(PASS)
pw.send_keys(Keys.CONTROL, 'v')
time.sleep(2)
driver.find_element_by_class_name('btn_login').click()

#while (driver.get_window_position)
'''pw = driver.find_element_by_id('pw')
pw.clear()
pw.send_keys(PASS)
captcha = driver.find_element_by_id('captcha')
captcha.clear()
captcha_answer = str(input("정답 : "))
captcha.send_keys(captcha_answer)
driver.find_element_by_class_name('btn_login').click()'''
time.sleep(1)
url_pay = "https://order.pay.naver.com/home"
driver.get(url_pay)
npoint = driver.find_element_by_css_selector(".my_npoint dd")
print(f"네이버포인트 : {npoint.text}")

C:\Users\user\AppData\Local\Temp\ipykernel_11156\290702224.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
C:\Users\user\AppData\Local\Temp\ipykernel_11156\290702224.py:8: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name('link_login').click()
C:\Users\user\AppData\Local\Temp\ipykernel_11156\290702224.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  id = driver.find_element_by_id('id')
C:\Users\user\AppData\Local\Temp\ipykernel_11156\290702224.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pw = driver.find_element_by_id('pw')
C:\Users\user\AppData\Local\Temp\ipykernel_11156\290702224.py:21: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLA

네이버포인트 : 556원


C:\Users\user\AppData\Local\Temp\ipykernel_11156\290702224.py:35: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  npoint = driver.find_element_by_css_selector(".my_npoint dd")


In [227]:
driver.quit()